In [1]:
# coding: utf-8
###=================================================###
###=================================================###
###     SOLVING LWR USING uSVP STRATEGY             ###
###=================================================###
###=================================================###

from timeit import default_timer as timer
import sys
import random
import time

#=====================================================

#============ BABAI'S NEAREST PLANE ALGORITHM=========

def coeff_x(xx,yy):
    """
    *Compute <xx,yy>/||yy||^2
    *Input:
            xx, yy: two vectors of same length
    *Output:
            <xx,yy>/||yy||^2        
    """
    return (xx.dot_product(yy))/(yy.dot_product(yy))
#--------------------

def Babai(AA, tt, lll_or_bkz20):
    """
    *Find a lattice vector of lattice L(AA) closest to target vector t (via Babai's Nearest Plane)
    *Input:
                    tt: target vector
                    AA: matrix of a lattice
        lll_or_bkz20=0: LLL
        lll_or_bkz20=1: BKZ20
    *Output: 
        closest_vector: vector of lattice L(AA) closest to tt
        
    """    
    
    vv=copy(tt)
    if lll_or_bkz20==0:                                      # using LLL or BKZ20 to reduce AA
        BB_lll=AA.LLL()
    if lll_or_bkz20==1:
        BB_lll=AA.BKZ(block_size=20)         
    BB=BB_lll.gram_schmidt()[0]
    m=BB.nrows()
    n=BB.ncols()
    closest_vector=vector(QQ, [0 for j in range(n)])           # closest_vector:=vetor 0
    i=m
    while i>0:
        i=i-1
        l_round=round(coeff_x(tt,BB[i]))
        tt=tt-l_round*BB_lll[i]
        closest_vector=closest_vector+l_round*BB_lll[i]
    return closest_vector
   
#==================GENERATING THE SECRET====================== 

def SecretGen2(qs,n):
    """
    *Generate the secret for LWR instance which is sampled uniformly at random from ZZ_qs
    *Input:
                qs: modulo for ZZ_qs 
                 n: the length of secret
    *Output:
              ssZZ: secret vector of length n <-- U(ZZ_qs)
    """
    
    ZZqs = IntegerModRing(qs)       # construct the ring ZZ_qs
    ss=random_vector(ZZqs,n)        # randomly generate the secret from ZZ_qs
    ssZZ=ss.change_ring(ZZ)          # change from ring ZZ_qs into ring ZZ
    return ssZZ

#--------------------------------
def SecretGen(qs,n):
    """
    *Generate the secret sampled uniformly at random from {-qs,...,qs} (e.g. qs=1 and s <--{-1,0,1} )
    *Input:
               qs: a (small) integer  
                n: the length of secret
    *Output: secret <---{-qs,...,qs} of length n
    """
    
    ssZZ=vector(ZZ, [randint(-qs, qs) for i in range(n)])
    return ssZZ

#==================GENERATING LWR INSTANCE==================== 

def round_modulo(aa, ss, q, p):
    """
    *Compute modulo rounding [<aa,ss>]_{q,p}
    *Input:
           aa, ss: vectors
             q, p:  modulus (q>p)
    *Output: the modulo rounding value [<aa,ss>]_{q,p}       
    """
    
    return round((p/q)*((aa.dot_product(ss))%q))%p
#--------------------
def LWR_Sampler(ss, q, p):
    """
    *Create a LWR sample of the form (aaZZ,c) where aaZZ is a vector, c is an integer
    *input:
              ss: the secret vector
            q, p: modulus (q>p)
    *Output:
       (aaZZ, c): an LWR sample        
    """
    n=len(ss)
    ZZq = IntegerModRing(q)                   # construct the ring ZZ_q
    aa=random_vector(ZZq,n)                   # random vector from ZZ_q
    aaZZ=aa.change_ring(ZZ)                   # change from ring ZZ_q into ring ZZ
    c=round_modulo(aaZZ, ss, q, p)            # modulo rounding
    return (aaZZ, c)
    
##--------------------------------
def LWR_Distribution(ss, m, q, p):
    """
    *Combine m LWR samples (aa,c) to the the pair (AALWR,ccLWR) where AA is an (m x n)-dimensional matrix 
    and cc is the vector consisting of rounded values c's in LWR samples.
    *Input:
                   ss:    the secret vector
                 q, p:    modulus (q>p)
                    m:    the number of LWR samples we want to have
    *Output:
        (AALWR, cLWR): LWR instance
        
    """
    
    n=len(ss)
    list_aaLWR=[]                # list of random vectors aa's
    list_cLWR=[]                 # list of c=round_modulo(aa,ss,q,p)
    for i in range (m):
        sample=LWR_Sampler(ss, q, p)
        aaLWR=sample[0]
        list_aaLWR.append(aaLWR)
        cLWR=sample[1]
        list_cLWR.append(cLWR)
    AALWR=matrix(ZZ, m, n, list_aaLWR)
    ccLWR=vector(ZZ, list_cLWR)
    return (AALWR, ccLWR)  
        
#======================= FINDING MODULO INVERSE MATRIX==================================
def modulo_inverse_matrix(AA, mod):
    """
    *Compute the inverse modulo mod of matrix AA
    *Input: 
                    AA:    matrix
                   mod:   modulo
    *Output:
        AA_inverse_mod: inverse modulo mod of AA 
    """

    det=AA.det()                                      # determinant of the matrix AA, i.e., det(AA)
    inverseAA=AA.inverse()                            # inverse of the matrix AA
    det_inverse=det.inverse_mod(mod)                  # inverse modulo mod of the real det(AA) 
    AA_inverse_mod=(det_inverse*det*inverseAA)%mod    # inverse modulo mod of AA
    
    return AA_inverse_mod

#==========================FINDING q-ARY BASIS=====================
def standard_qAry_basis(AA, mod):
    """
    *Compute a basis for q-ary lattice generated by matrix AA in standard uSVP and BDD strategy
    *Input:
                AA:   matrix
               mod:   modulo
    *Output:
        qary_basis: q-ary basis 
    """
    
    m=AA.nrows()
    qII=mod*identity_matrix(m)         # the matrix mod*II_m where II_m is the identity matrix) 
    BB=qII.stack(AA.transpose())       # Create concatenation matrix of the form (qII   AA)^T
    BB_HNF=BB.echelon_form()           # compute Hermite Normal Form of matrix BB
    qary_basis=BB_HNF[0:m]             # m first rows of BB_HNF (which is non-zero)
    return qary_basis


#==========================REDUCING LWR TO LWE BEFORE ATTACKING ==================

#-------REDUCING LWR TO LWE MODULO q---------

def LWR_to_LWE_modq(AALWR, ccLWR, ss, q, p):
    """
    *Reduce LWR to LWE mod q
    *Secret ss is just used to compute error eeLWEmodq
    *Input:
                           ss:    secret
               (AALWR, ccLWR):    LWR instance
                         q, p:    modulus
    *Output: 
        (AALWEmodq, ccLWEmodq: LWE mod q insatance. Note, AALWEmodq=AALWR, ccLWEmodq=round((q/p)*ccLWR)
                    eeLWEmodq: LWE mod q error
    """
    
    AALWR=AALWR.change_ring(ZZ)
    AALWEmodq=AALWR
    m=AALWR.nrows()
    ccLWEmodq=vector(ZZ,m)
    cc=(AALWR*ss)%q                                
    for j in range(m):
        ccLWEmodq[j]=round((q/p)*ccLWR[j])
    eeLWEmodq=ccLWEmodq-cc                         # error is induced from LWR to LWE mod q
    return (AALWEmodq, ccLWEmodq, eeLWEmodq)

#-------REDUCING LWR TO LWE MODULO q_new---------

def LWR_to_LWE_modq_new(AALWR, ccLWR, ss, q, p, q_new):
    """
    *Reducing LWR to LWE mod q_new
    *Parameters:
                ss:    secret
    (AALWR, ccLWR):    LWR instance
              q, p:    modulus (q>p)
             q_new:    new modulo
    *Output: 
        (AALWEmodq_new, ccLWEmodq_new): LWE mod q_new instance.
         Note, AALWEmodq_new=round((q_new/q)AALWR), ccLWEmodq=round((q_new/p)*ccLWR)
                         eeLWEmodq_new: LWE mod q_new error
    """
    
    list_AA=[]
    m=AALWR.nrows()
    n=AALWR.ncols()
    ccLWEmodq_new=vector(ZZ,m)
    for j in range(m):
        ccLWEmodq_new[j]=round((q_new/p)*ccLWR[j])      #ccLWEmodq=round((q_new/p)*ccLWR)
    for aaLWR in AALWR:
        aaLWRZZ=aaLWR.change_ring(ZZ)
        aaLWEmodq_new=vector(ZZ,n)
        for j in range(n):
            aaLWEmodq_new[j]=round((q_new/q)*aaLWRZZ[j]) #AALWEmodq_new=round((q_new/q)AALWR)
        list_AA.append(aaLWEmodq_new) 
    AALWEmodq_new=matrix(ZZ, m, n, list_AA)               
    cc=(AALWEmodq_new*ss)%q_new      
    eeLWEmodq_new=ccLWEmodq_new-cc                       # error is induced from LWR to LWE mod q_new
    return (AALWEmodq_new, ccLWEmodq_new, eeLWEmodq_new)

#================================BDD STRATEGY==============================

def BDD_Solver(AA, cc, ss, mod, lll_or_bkz20):
    """
    *Solving LWE via BDD using LLL
    *Input: 
              (AA, cc): LWE instance 
                    ss: secret vector
                   mod: modulo
        lll_or_bkz20=0: LLL
        lll_or_bkz20=1: BKZ20
    *Output: 
               solvabl: True/False saying that LWE is solvable or not 
         solvable=True: We succeed
        solvable=False: We fail
    """
    
    BB=standard_qAry_basis(AA,mod)
    uu=Babai(BB, cc, lll_or_bkz20)
    n=AA.ncols()
    subAA=matrix(ZZ,n)
    det=subAA.det()
    #This 'while loop' is used to find some sub matrix subAA of AA is invertible modulo mod, i.e. gcd(det(subAA), mod)=1. 
    #To do that we choose randomly n vectors of AA to create subAA and check gcd(det(subAA), mod)=1 or not. If not, we do again
    while gcd(det,mod)!=1:
        i=0
        list_vector_of_AA=list(AA)
        list_element_of_uu=list(uu)
        list_vector_of_subAA=[]
        list_element_of_sub_uu=[]
        while i<n:
            t1=random.choice(list_vector_of_AA)
            index=list_vector_of_AA.index(t1)
            list_vector_of_subAA.append(t1)
            t2=list_element_of_uu[index]
            list_vector_of_AA.remove(t1)
            list_element_of_uu.remove(t2)
            list_element_of_sub_uu.append(t2)
            i=i+1
        sub_uu=vector(ZZ,list_element_of_sub_uu)    #sub_uu: a sub-vector of uu such that sub_uu=subAA*ss modulo mod
        subAA=matrix(ZZ,list_vector_of_subAA)
        det=subAA.det()
    inverse_of_subAA=modulo_inverse_matrix(subAA,mod)
    ss_recover=(inverse_of_subAA*sub_uu)%mod
    print "*The original secret ss=",ss
    print "*The recovered secret ssrecover=", ss_recover
    sovable=False
    if ss%mod==ss_recover or ss==ss_recover:
        sovable=True
    if sovable==True:
        print "SUCCESSFUL"
    else: 
        print "FAIL"  
    return sovable       
           
        
 #==============================STANDARD uSVP STRATEGY==============================

def uSVP_Standard_matrix(AA, cc, t):
    """
    *Create standard uSVP matrix 
    *Input:
             (AA,cc): LWE instance
                   t: embedding factor integer (typically, t=1)
    *Output: 
                  BB: of the form (AA cc)
                                  (00  t)   
    """
    
    AA1=AA.augment(cc)                                             # create AA1=(AA cc)
    tt=vector(list(vector([0 for i in range(AA.ncols())]))+[t])    # creat tt=(00 t)
    BB=AA1.stack(tt)                                               # stack AA1 over tt 
    return BB 
    

#------------------------------------
def uSVP_Standard_Solver(AA, cc, ss, ee, t, mod, lll_or_bkz20):
    """
    *Solving LWE via uSVP standard using LLL or BKZ20
    *Input: 
              (AA, cc): LWE instance
                    ss: secret vector
                    ee: error (for check the result)
             algorithm: integer 0 or 1
       lll_or_bkz20==0: choose LLL           
       lll_or_bkz20==1: choose BKZ20
    *Output: 
              solvable: True/False saying that LWE is solvable or not 
         solvable=True: We succeed
        solvable=False: We fail
    """        
    n=AA.ncols()
    BA=uSVP_Standard_matrix(AA, cc, t)  		# construct uSVP matrix  
    BB=standard_qAry_basis(BA, mod)                     # find q-ary basis modulo mod
    if lll_or_bkz20==0:                                    # using LLL or BKZ20 to reduce BB
        BB_reduced=BB.LLL()
    if lll_or_bkz20==1:
        BB_reduced=BB.BKZ(block_size=20)         
    List_of_candidates=[]                                          # list of vectors which have its last element =t or -t
    for bb_reduced in BB_reduced:
        if bb_reduced[-1]==t: 
            List_of_candidates.append(bb_reduced)
        if bb_reduced[-1]==-t:
             List_of_candidates.append(-bb_reduced)
    solvable=False                                         #used to check if we can solve uSVP
    for bb_reduced in List_of_candidates:                              
        ee_reduced=vector(ZZ,[bb_reduced[i] for i in range(len(bb_reduced)-1)]) 
        uu=(cc-ee_reduced)%mod                     #compute uu:=cc-ee_reduced (=AA*ss modulo mod)
        subAA=matrix(ZZ,n)                            #subAA: a square sub-matrix of AA
        det=subAA.det()
    
#This 'while loop' is used to find some sub matrix subAA of AA is invertible modulo mod, i.e. gcd(det(subAA), mod)=1.#
#To do that we choose randomly n vectors of AA to create subAA and check gcd(det(subAA), mod)=1 or not. If not, we do again#
        while gcd(det, mod)!=1:                     
            i=0
            list_vector_of_AA=list(AA)
            list_element_of_uu=list(uu)
            list_vector_of_subAA=[]
            list_element_of_sub_uu=[]             #sub_uu: a sub-vector of uu such that sub_uu=subAA*ss modulo mod
            while i<n:
                t1=random.choice(list_vector_of_AA)
                index=list_vector_of_AA.index(t1)
                list_vector_of_subAA.append(t1)
                t2=list_element_of_uu[index]
                list_vector_of_AA.remove(t1)
                list_element_of_uu.remove(t2)
               
                list_element_of_sub_uu.append(t2)
                i=i+1
            subAA=matrix(ZZ, list_vector_of_subAA)
            sub_uu=vector(ZZ, list_element_of_sub_uu)
            det=subAA.det()
        inverse_of_subAA=modulo_inverse_matrix(subAA, mod)    	 #Compute inverse modulo of subAA
        ss_recover=(inverse_of_subAA*sub_uu)%mod		 #recover secret vector
        if ss%mod==ss_recover:
            solvable=True
        break    
    if solvable==True:
        print "SUCCESSFUL"
    if solvable==False: 
        print "FAIL"
    return solvable      
      
    #==============================uSVP DUAL STRATEGY==============================
            
def uSVP_Dual_matrix(AA, cc):
    """
    *Create dual uSVP matrix of form  (AA  II_m  cc), m=number of rows of AA
    *Input: 
                (AA, cc): LWE instance
    *Output:  
        matrix_dual_uSVP: of the form (AA  II_m  cc)
    """
    
    m=AA.nrows()
    II_m=identity_matrix(ZZ,m)                #II_m: (m x m)-identity matrix 
    AA=AA.augment(II_m)                        #AA---> (AA   II_m)
    matrix_dual_uSVP=AA.augment(cc)             #(AA   II_m) ---> (AA  II_m  cc)
    return matrix_dual_uSVP 
    
#------------------------------------
def qAryBasis_Dual_uSVP(AA, mod):   
    """
    *Find q-ary basis of q-ary lattice generated by AA
    *Input:
                    AA: LWE matrix
                   mod: modulo
    *Output: 
        dual_qAry_basis: dual q-ary basis
    """
    m=AA.nrows()
    n=AA.ncols()
    modII_n=mod*identity_matrix(n)
    BB=AA.stack(modII_n)                             # Creat BB=(AA   mod*II_n)^T
    left_kernel_BB=BB.left_kernel()                   # left kernel of BB 
    BB_HNF=left_kernel_BB.echelonized_basis()          # Hermite Normal Form of left_kernel_BB
    list_of_vectors=[]
    for bb in BB_HNF:
        list_of_vectors.append(vector(ZZ,[bb[j] for j in range(m)]))
    dual_qAry_basis=matrix(list_of_vectors)
    return dual_qAry_basis

#------------------------------------
def uSVP_Dual_Solver(AA, cc, ss, mod, lll_or_bkz20):
    """
    Solving LWE via dual uSVP using LLL or BKZ20
         (AA, cc): LWE instance
                  ss: secret vector
                  ee: error (for check the result)
        lll_or_bkz20: integer 0 or 1
     lll_or_bkz20==0: choose LLL           
     lll_or_bkz20==1: choose BKZ20
    """
    
    n=AA.ncols()
    BA=uSVP_Dual_matrix(AA, cc)
    BB=qAryBasis_Dual_uSVP(BA.transpose(),mod)
    if lll_or_bkz20==0:                                      # using LLL or BKZ20 to reduce BB
        BB_reduced=BB.LLL()
    if lll_or_bkz20==1:
        BB_reduced=BB.BKZ(block_size=20)         
    List_of_candidates=[]                                     # list of vectors which have its last element =-t
    for bb_reduced in BB_reduced:
        if bb_reduced[-1]==-1: 
            List_of_candidates.append(bb_reduced)
    solvable=False
    for short in List_of_candidates:                                   #short has form of (ss | ee | -1)
        ss_recover=vector([short[i] for i in range(n)])
        print "*The original secret ss=",ss
        print "*The recovered secret ssrecover=", ss_recover
        if ss==ss_recover:
            solvable=True
        break
    if solvable==True:
        print "SUCCESSFUL"
    else: 
        print "FAIL"
    return solvable        
           

#======================= SOLVABLE RANGE===================
def choose_q_min(n, alpha, lll_or_bkz20):
    """
    *Choose q_min, given alpha such that LWR is solvable.
    *Note: replace c0 by cLLL or cBKZ
    *Input:
                       n: dimension of LWR problem
                   alpha: the bit-ratio between q and p
         lll_or_bkz20==0: use LLL ---> cLLL 
         lll_or_bkz20==1: use BKZ(200) ---> cBKZ
    *Output:
       bit_size_of_q_min: bit size of minimal q such that LWR cab R is solvable. 
    """
    if lll_or_bkz20==0:
        c0=cLLL
    if lll_or_bkz20==1:
        c0=cBKZ
    bit_size_of_q_min=ceil(-4*n*log(c0,2)/(alpha^2))
    return bit_size_of_q_min
    
    
    
#=================================

def print_notation():
    """
    print notations used in output part
    """
    print "-------------------------------------------NOTATIONS---------------------------------------------"
    print "                n: dimension of LWR problem"
    print "            alpha: the bit-ratio between q and p"
    print "           rq_min: the lower bound of solvable range of LWR problem given n, LLL or BKZ used"
    print "               rq: the chosen bit-size of q (we set rq=rq_solvable_min+2)"
    print "               rp: the chosen bit-size of p (we set r=round(rq*alpha))"
    print "         r_newopt: the maximum bit-size of q_new that we can use for modular q_new approach"
    print "            r_new: the real bit-size of q_new that we use for modular q_new approach (we set r_new=rq+2)"
    print "                q: the original modulus q (i.e. q=2^(rq))"
    print "                p: the rounding modulus p of the LWR problem (i.e. p=2^(rp))"
    print "            q_new: the modulus that we use for our approach (i.e. q_new=next_prime(2^(r_new)))"
    print "                m: the optimal number of samples for modular q approach w.r.t the tuple of parameters"
    print "               qs: the real size of secret key (i.e., we set s<----{-qs,...., qs}"
    print "number_of_running: the number of times that we run for the LWR instance"
   

#==============GENERATING PARAMETERS FOR ATTACKS===================

def generate(n, alpha, number, k, l, qs, lll_or_bkz20):
    """
    *Generating parameters for attacks
    * Input: 
                      n: dimension
                  alpha: bit-size-of-p/ bit-size-of-q (0<alpha<=1)
      number_of_running: number of running for each LWR instance
                      k: real bit-size-of-q=minimum bit-size-of-q + k 
                      l: real bit-size-of-q_new=optimal bit-size-of-q_new + l
                     qs: for secret ZZ_qs or {-qs,.., qs}
        lll_or_bkz20==0: use LLL and cLLL 
        lll_or_bkz20==1: use BKZ(200) and cBKZ
    """
    if lll_or_bkz20==0:
        c0=cLLL
    if lll_or_bkz20==1:
        c0=cBKZ
           
    rq_min=choose_q_min(n, alpha, lll_or_bkz20)        #minimum bit-size-of-q
    rq=rq_min+k                              # real bit-size-of-q
    rp=round(rq*alpha)			     #bit-size-of-p
    q=2^rq                                   # q, p =power(2)
    p=2^rp
    m=round(sqrt(-n*log(q,2)/(log(c0,2))))-n    # number of LWR samples
    sigma_s=((2*qs+1)^2-1)/12                    
    C=q^2*p^2+n*sigma_s^2*p^2*q^2
    B=q^2+2*q
    
    N=n*sigma_s^2+1
    M=(q^2+2*q)/(q^2*p^2)
    
    q_newopt=sqrt((m+12)*N/((n+1)*M))             #optimal q_new
    
    q_newopt1=sqrt((m+12)*N/((n+1)*M))             #optimal q_new
    q_newopt2=sqrt((N-1)/M)             #optimal q_new
    
    #q_newopt=sqrt((m-n)*C/(n*B))
    
    r_newopt=RR(log(q_newopt,2))             #optimal bit-size-of-q_new
    
    r_newopt1=RR(log(q_newopt1,2))             #optimal bit-size-of-q_new
    r_newopt2=RR(log(q_newopt2,2))             #optimal bit-size-of-q_new
    r_new= r_newopt+l                        #real bit-size-of-q_new
    q_new= next_prime(round(2^(r_new)))
    
    print "r_newopt1=", r_newopt1
    print "r_newopt2=", r_newopt2
    
    print "(n, alpha, rq_min, rq, rp, r_newopt, r_new, p, q, q_new, m, s in {-1,0,1} (qs=1), number of running)="
    print (n, alpha, rq_min, rq, rp, r_newopt, r_new, p, q, q_new, m, qs, number_of_running)
    return (n, alpha, rq_min, rq, rp, r_newopt, r_new, p, q, q_new, m, qs, number_of_running)
    

#==============MAIN PROCEDURE===================
#======================================
def solving_result(n, alpha, number_of_running, qs, lll_or_bkz20):

   for k in [0]:
        for l in [0]:
            gen=generate(n, alpha, number_of_running, k, l, qs, lll_or_bkz20)
            n=gen[0]
            alpha=gen[1]
            rq_solvable_min=gen[2]
            rq=gen[3]
            rp=gen[4]
            
            r_newopt=gen[5]
            r_new=gen[6]
            p=gen[7]
            q=gen[8]
            q_new=gen[9]
            m=gen[10]
            qs=gen[11]
            number_of_running=gen[12]
           
            
            #count the number of success
            count1=0   
            count2=0
            count3=0
            count4=0
            count5=0   
            count6=0
            count7=0
            count8=0
            count9=0
            
            # save time information
            Time_list1=[]
            Time_list2=[]
            Time_list3=[]
            Time_list4=[]
            Time_list5=[]
            Time_list6=[]
            Time_list7=[]
            Time_list8=[]
            Time_list9=[]
            
            
  
            for tt in range(0, number_of_running):
                print "THE", tt, "th CASE:"
                #--------------KEY GENERATION--------------
                ss=SecretGen(qs, n)
                #--------------LWR SAMPLES--------------
                sample=LWR_Distribution(ss, m, q, p)
                AALWR=sample[0]
                ccLWR=sample[1]
                #---------MODULAR q APPROACH'94----------
                LWEmodq=LWR_to_LWE_modq(AALWR, ccLWR, ss, q, p)
                AALWEmodq=LWEmodq[0]
                ccLWEmodq=LWEmodq[1]
                eeLWEmodq=LWEmodq[2]
                
                #--------------MODULAR q_new APPROACH'94--------------
                LWEmodq_new=LWR_to_LWE_modq_new(AALWR, ccLWR, ss, q, p, q_new)
                AALWEmodq_new=LWEmodq_new[0]
                ccLWEmodq_new=LWEmodq_new[1]
                eeLWEmodq_new=LWEmodq_new[2]
                                
                
                #print "================SOLVING via BDD==========================="
                #print "---------------"
                #print "BDD mod q:"
                #print "---------------"
                start1=timer()
                #sovable1=BDD_Solver(AALWR, ccLWEmodq, ss, q, lll_or_bkz20)
                #if sovable1==True:
                    #count1=count1+1
                end1=timer()
                #print "*The running time is", end1-start1 , " sec"
                #Time_list1.append(end1-start1)
                
                #print "---------------"
                #print "BDD mod q_new:"
                #print "---------------"
                start7=timer()
                #sovable7=BDD_Solver(AALWEmodq_new, ccLWEmodq_new, ss, q_new, lll_or_bkz20)
                #if sovable7==True:
                    #count7=count7+1
                #end7=timer()
                #print "*The running time is", end7-start7 , " sec"

                #Time_list7.append(end7-start7)
             
                #print "============================SOLVING via STANDARD uSVP====================="
                #print "---------------------"
                #print "Standard uSVP mod q:"
                #print "---------------------"
                #start3=timer()
                #solvable3=uSVP_Standard_Solver(AALWEmodq, ccLWEmodq, ss, eeLWEmodq, 1, q, lll_or_bkz20)
                #if solvable3==True:
                    #count3=count3+1
                #end3=timer()
                #print "*The running time is", end3-start3 , " sec"
                #Time_list3.append(end3-start3)
 
                #print "---------------------"
                #print "Standard uSVP mod q_new:"
                #print "---------------------"
                #start8=timer()
                #solvable8=uSVP_Standard_Solver(AALWEmodq_new, ccLWEmodq_new, ss, eeLWEmodq_new, 1, q_new, lll_or_bkz20)
                #if solvable8==True:
                    #count8=count8+1
                #end8=timer()
                #print "*The running time is", end8-start8 , " sec"
                #Time_list8.append(end8-start8)
                
                print "========================SOLVING via DUAL uSVP=========================="
        	print "---------------------"
                print "Dual uSVP mod q:"
                print "---------------------"
                start5=timer()
                
                sovable5=uSVP_Dual_Solver(AALWEmodq, ccLWEmodq, ss, q, lll_or_bkz20)
                if sovable5==True:
                    count5=count5+1
                end5=timer()
                print "*The running time is", end5-start5 , " sec"
                
                Time_list5.append(end5-start5)
   	        print "---------------------"
                print "Dual uSVP mod q_new:"
                print "---------------------"
                start9=timer()           
                solvable9=uSVP_Dual_Solver(AALWEmodq_new, ccLWEmodq_new, ss, q_new, lll_or_bkz20)
                if solvable9==True:
                    count9=count9+1
                end9=timer()
                print "*The running time is", end9-start9 , " sec"
                Time_list9.append(end9-start9)
                print"========================================="
           
                
        print "                   SUMMARY:"
                  
        #print "The ratio of success of BDD (LLL) mod q=            ", count1,"/", number_of_running
                
        #print "The ratio of success of BDD (LLL) mod q_new=        ", count7,"/", number_of_running
    
        #print "The ratio of success of uSVP Standard (LLL) mod q=    ", count3,"/", number_of_running
                
        #print "The ratio of success of uSVP Standard (LLL) mod q_new=", count8,"/", number_of_running
   
        print "The ratio of success of uSVP Dual (LLL) mod q=        ", count5,"/", number_of_running
                
        print "The ratio of success of uSVP Dual (LLL) mod q_new=    ", count9, "/", number_of_running
                
                
        #print "List of time BDD (LLL) mod q=", Time_list1
                
        #print "List of time BDD (LLL) mod q_new=", Time_list7
                
        #print "List of time uSVP Standard (LLL) mod q=", Time_list3
                
        #print "List of time uSVP Standard (LLL) mod q_new=", Time_list8
     
        print "List of time uSVP Dual (LLL) mod q=", Time_list5
                
        print "List of time uSVP Dual (LLL) mod q_new=", Time_list9
                    
#=========--------------------INPUT----------------------==================

dimension_n=20             
bit_ratio_q_p=1/2       
number_of_running=3
qs=1
lll_or_bkz20 =0 # if you choose LLL
#lll_or_bkz20 =1 #if you choose BKZ(20)



#=====================Constants=================
cLLL=0.9775
cBKZ=0.9863
#===================OUTPUT==============

print_notation()
print"-------------------------------------------RESULTS---------------------------------------------"

solving_result(dimension_n, bit_ratio_q_p, number_of_running, qs, lll_or_bkz20)

print "-----------"

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("*The original secret ss=",ss)? (<ipython-input-1-a3823d28dbc1>, line 285)